In [1]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import math
from timeit import default_timer as timer
from datetime import datetime, timedelta
import numba
from xgboost_general_symbol_ensemble_sharecast import load_data

LABEL_COLUMN = "Future8WeekReturn"

/Users/neilkloot/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
pd.options.display.max_rows=500

In [3]:
# df1 = pd.read_pickle('../data/ml-20180512-processed.pkl.gz', compression='gzip')
# df2 = pd.read_pickle('../data/ml-20180512-labelled.pkl.gz', compression='gzip')
# df1 =  load_data('../data/ml-20180512-processed.pkl.gz', False)

df1 = load_data('../data/ml-20180512-processed.pkl.gz',
                               drop_unlabelled=False,
                               drop_labelled=True,
                               generate_labels=False,
                               label_weeks=8,
                               reference_date='2018-05-12')

Loading file: ../data/ml-20180512-processed.pkl.gz
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3332090 entries, 2007-07-02 to 2018-05-11
Columns: 224 entries, symbol to H05_GLFSEPTPOP_T11_20P
dtypes: category(15), datetime64[ns](9), float32(129), float64(63), int32(1), int8(7)
memory usage: 3.5 GB
None
Converting dates to datetime types
Dropping columns: ['Price200DayAverage', 'Price52WeekPercChange', 'AverageVolume', 'EBITDMargin', 'EPSGrowthRate10Years', 'EPSGrowthRate5Years', 'IAD', 'LTDebtToEquityQuarter', 'LTDebtToEquityYear', 'NetIncomeGrowthRate5Years', 'NetProfitMarginPercent', 'PriceToBook', 'ReturnOnAssets5Years', 'ReturnOnAssetsTTM', 'ReturnOnAssetsYear', 'ReturnOnEquity5Years', 'ReturnOnEquityTTM', 'RevenueGrowthRate10Years', 'RevenueGrowthRate5Years', 'TotalDebtToAssetsQuarter', 'TotalDebtToAssetsYear', 'TotalDebtToEquityQuarter', 'bookValue', 'earningsPerShare', 'ebitda', 'epsEstimateCurrentYear', 'marketCapitalization', 'peRatio', 'pegRatio', 'pricePerBook', 'pr

In [4]:
df1.columns.values

array(['symbol', 'lastTradePriceOnly', 'adjustedPrice', 'volume',
       'daysHigh', 'daysLow', 'previousClose', 'change',
       'changeInPercent', '52WeekHigh', '52WeekLow',
       'changeFrom52WeekHigh', 'changeFrom52WeekLow',
       'percebtChangeFrom52WeekHigh', 'percentChangeFrom52WeekLow',
       'allordpreviousclose', 'allordchange', 'allorddayshigh',
       'allorddayslow', 'allordpercebtChangeFrom52WeekHigh',
       'allordpercentChangeFrom52WeekLow', 'asxpreviousclose',
       'asxchange', 'asxdayshigh', 'asxdayslow',
       'asxpercebtChangeFrom52WeekHigh', 'asxpercentChangeFrom52WeekLow',
       'exDividendPayout', '640106_A3597525W', 'AINTCOV', 'Beta',
       'BookValuePerShareYear', 'CashPerShareYear', 'DPSRecentYear',
       'EPS', 'FIRMMCRT', 'FXRUSD', 'Float', 'GRCPAIAD', 'GRCPAISAD',
       'GRCPBCAD', 'GRCPBCSAD', 'GRCPBMAD', 'GRCPNRAD', 'GRCPRCAD',
       'H01_GGDPCVGDP', 'H01_GGDPCVGDPFY', 'H05_GLFSEPTPOP', 'MarketCap',
       'OperatingMargin', 'PE', 'QuoteLast',

In [5]:
# df1[['quoteDate_YEAR', 'quoteDate_MONTH', 'quoteDate_DAY']]
# pd.to_datetime(date_df)

In [6]:
from execute_xgboost_predictions import prepare_data_for_model

df_all_x, df_symbol_date = prepare_data_for_model(df1, False)

No of symbols: 1721
Symbol: LNY number of records: 29


/Users/neilkloot/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/neilkloot/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Symbol: IAA number of records: 37
Symbol: GNE number of records: 37
Symbol: CCA number of records: 37
Symbol: IMF number of records: 36
Symbol: REX number of records: 34
Symbol: XPE number of records: 37
Symbol: AMP number of records: 37
Symbol: MA1 number of records: 33
Symbol: RVR number of records: 36
Symbol: IEL number of records: 36
Symbol: WLD number of records: 34
Symbol: COM number of records: 19
Symbol: FMG number of records: 35
Symbol: PFG number of records: 26
Symbol: TBR number of records: 21
Symbol: PBT number of records: 30
Symbol: AOU number of records: 27
Symbol: LGR number of records: 20
Symbol: BYI number of records: 3
Symbol: USA number of records: 22
Symbol: RIR number of records: 5
Symbol: CDH number of records: 1
Symbol: 1ST number of records: 18
Symbol: RXP number of records: 36
Symbol: AO1 number of records: 14
Symbol: BUD number of records: 24
Symbol: EBG number of records: 8
Symbol: RHL number of records: 37
Symbol: AYS number of records: 37
Symbol: TCN number

In [7]:
df_symbol_date

,symbol
quoteDate_ref,
2018-03-19,LNY
2018-03-20,LNY
2018-03-22,LNY
2018-03-26,LNY
2018-03-27,LNY
2018-03-28,LNY
2018-04-03,LNY
2018-04-04,LNY
2018-04-06,LNY


In [8]:
df_symbol_date.to_csv('test.csv')